# Paper Data Prep

Factor data:
- 25 size-value FF portfolios 
- FF Industry portfolios
- the 6 portfolios which form SMB and HML
    - sourced from Ken French's website
- DEF (difference between the return on long-term corporate bonds and long-term government bonds)
- TERM (difference between the return on 30 year government bonds and the short-term rate)
    - both sourced Ibbotson
    - possible sub for corp bond returns (https://fred.stlouisfed.org/series/BAMLCC8A015PYTRIV)
    - CRSP 20 or 30 year bond return, short term as 30 or 90 day
- DEFY (default yield spread: Moody's BAA and AAA yield spread)
    - Amit Goyal or FRED (https://fred.stlouisfed.org/series/BAA & https://fred.stlouisfed.org/series/AAA)
- TERMY (term yield spread: 10 year and 1 year Treasury spread)
    - FRED (https://fred.stlouisfed.org/series/DGS10 & https://fred.stlouisfed.org/series/DGS1)
- RF (30 day T-bill rate)
    - CRSP
- Market portfolio (value-weighted NYSE)
- GDP (seasonally adjusted)
    - FRED (https://fred.stlouisfed.org/series/GDPC1)

Other required data:
- ISM Manufacturing Index
    - sourced from Bloomberg
- FF5 factors + Momentum
    - Ken French
- BBK Monthly GDP
    - FRED (https://fred.stlouisfed.org/series/BBKMGDP)

In [128]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
import pandas_datareader.data as pdr

## FRED Data

In [129]:
fred_data_m = pdr.DataReader(['BAA', 'AAA'],'fred', start='1947-01-01')
fred_data_m.index = fred_data_m.index + MonthEnd(0)

In [130]:
fred_data_d = pdr.DataReader(['DGS10' ,'DGS1', 'BAMLCC8A015PYTRIV'],'fred', start='1947-01-01')
fred_data_d = fred_data_d.rename(columns={'DGS10':'10 year' ,'DGS1':'1 year', 'BAMLCC8A015PYTRIV':'corp_bond_return'})

# Resample to end of month values
fred_data_d_m = fred_data_d.resample('M').last()

In [132]:
fred_data = pd.merge(fred_data_m, fred_data_d_m, how='outer', left_index=True, right_index=True)
fred_data.head()

,BAA,AAA,10 year,1 year,corp_bond_return
DATE,,,,,
1947-01-31,3.13,2.57,NaN,NaN,NaN
1947-02-28,3.12,2.55,NaN,NaN,NaN
1947-03-31,3.15,2.55,NaN,NaN,NaN
1947-04-30,3.16,2.53,NaN,NaN,NaN
1947-05-31,3.17,2.53,NaN,NaN,NaN


#### GDP

In [66]:
gdp_data = pdr.DataReader(['GDPC1'],'fred', start='1947-01-01')
gdp_data.index = gdp_data.index + MonthEnd(0)
gdp_data.head()

,GDPC1
DATE,
1947-01-31,2034.450
1947-04-30,2029.024
1947-07-31,2024.834
1947-10-31,2056.508
1948-01-31,2087.442


### BBK GDP (and CPI)
The Brave-Butters-Kelley Monthly Real GDP Series is a monthly estimate of real GDP using a dynamic factor model of nearly 500 mixed frequency economic indicators. It can be interpreted as an estimate of the GDP growth of a particular month.

In [173]:
bbk_data = pdr.DataReader(['BBKMGDP', 'CPIAUCSL'],'fred', start='1947-01-01')
bbk_data.index = bbk_data.index + MonthEnd(0)
bbk_columns = ['gdp', 'cpi']
bbk_data.head()

,BBKMGDP,CPIAUCSL
DATE,,
1947-01-31,NaN,21.48
1947-02-28,NaN,21.62
1947-03-31,NaN,22.00
1947-04-30,NaN,22.00
1947-05-31,NaN,21.95


## Ken French Data

In [185]:
from pandas_datareader.famafrench import get_available_datasets
# get_available_datasets()

In [69]:
def get_ff_data():
    series = ['F-F_Research_Data_5_Factors_2x3', '25_Portfolios_5x5', 
                          'F-F_Momentum_Factor', '30_Industry_Portfolios']
    
    dataframes = [pdr.DataReader(data,'famafrench', start='1925-01-01')[0] for data in series]
    
    df = pd.concat(dataframes, axis=1).sort_values(by='Date')
    df.index = df.index.to_timestamp() + MonthEnd(0)
    
    return df

In [70]:
ff_data = get_ff_data()

In [71]:
ff_data.tail()

,Mkt-RF,SMB,HML,RMW,CMA,RF,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,...,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30,-9.35,-0.97,0.06,-1.51,-0.84,0.19,-13.1838,-9.2149,-7.2748,-11.4204,...,-13.94,-11.07,-11.54,-13.27,-14.24,-9.46,-7.67,-6.26,-7.73,-6.40
2022-10-31,7.83,1.86,8.05,3.07,6.52,0.23,3.2654,5.1384,8.7991,9.0880,...,10.94,1.99,8.97,10.02,6.68,13.65,1.94,10.26,12.80,11.25
2022-11-30,4.60,-2.67,1.38,6.01,3.11,0.29,-5.6998,-3.0675,0.6161,0.9805,...,2.32,5.66,4.93,6.96,10.32,5.05,2.95,5.65,4.75,6.54
2022-12-31,-6.41,-0.16,1.32,0.09,4.19,0.33,-6.4639,-4.6920,-5.2707,-4.8792,...,-6.76,-6.68,-9.07,-4.09,-7.66,-5.63,-8.97,-6.82,-5.49,-3.06
2023-01-31,6.65,4.43,-4.08,-2.61,-4.52,0.35,16.8009,14.5972,13.9285,9.3206,...,13.45,9.76,9.81,3.27,8.55,6.89,10.92,10.48,6.46,1.74


In [181]:
# ff_data.to_csv("ff_data.csv")

In [188]:
ff_pred_data = pdr.DataReader('6_Portfolios_2x3','famafrench', start='1925-01-01')[0]
ff_pred_data.index = ff_pred_data.index.to_timestamp() + MonthEnd(0)
ff_pred_data.columns = ['SL', 'SM', 'SH', 'BL', 'HM', 'BH']

In [189]:
ff_pred_data.head()

,SL,SM,SH,BL,HM,BH
Date,,,,,,
1926-07-31,1.0874,0.9081,-0.0695,5.7168,1.8971,2.0066
1926-08-31,0.7030,1.5075,5.3842,2.7154,2.7196,5.6796
1926-09-30,-2.9117,-0.1359,-0.4374,1.4287,0.0808,-0.7928
1926-10-31,-3.8196,-4.3572,-2.0112,-3.5898,-2.3377,-3.9998
1926-11-30,3.1806,3.6608,2.0944,3.1292,2.9242,3.1934


## CRSP Data

In [165]:
crsp_data.columns

Index(['CRSP 20-Year Bond Returns', 'CRSP 30-Day Bill Returns',
       'CRSP 30-Year Bond Returns', 'CRSP 90-Day Bill Returns',
       'CRSP NYSE Value-Weighted Market Index'],
      dtype='object', name='IndNm')

In [166]:
crsp_data = pd.read_csv("crsp_data.csv", index_col='MthCalDt', parse_dates=True)
crsp_data = crsp_data.pivot(columns="IndNm", values='COL1')
crsp_data.index = crsp_data.index + MonthEnd(0)
crsp_data.columns = ['20y_return', 'rf', '30y_return', '90day', 'mkt']
crsp_data.index.name = 'Date'

In [167]:
crsp_data.tail()

,20y_return,rf,30y_return,90day,mkt
Date,,,,,
2022-08-31,-0.046849,0.001901,-0.053490,0.001806,-0.030011
2022-09-30,-0.067945,0.001929,-0.085155,0.001684,-0.085177
2022-10-31,-0.041092,0.002327,-0.065624,0.001859,0.106952
2022-11-30,0.071208,0.002856,0.077931,0.003114,0.058064
2022-12-31,-0.025772,0.003379,-0.035953,0.003280,-0.040356


## ISM

In [74]:
# ISM Quarterly
ism_data = pd.read_excel("ism_data.xlsx")
ism_data = ism_data.set_index('Date', drop=True).sort_values(by='Date')
ism_data.columns = ["ISM"]

In [75]:
ism_data.head()

,ISM
Date,
1948-03-31,43.3
1948-06-30,53.0
1948-09-30,42.1
1948-12-31,35.0
1949-03-31,34.5


In [89]:
# ISM Monthly
ism_monthly = pd.read_excel("ism_monthly.xlsx", index_col=0, parse_dates=True)
ism_monthly = ism_monthly.sort_values(by='Date')
ism_monthly.head()

,ISM
Date,
1948-01-31,51.7
1948-02-29,50.2
1948-03-31,43.3
1948-04-30,45.4
1948-05-31,49.5


## Merging

### Monthly Data

In [190]:
data = [fred_data, bbk_data, crsp_data, ism_monthly, ff_pred_data]

In [191]:
merged = pd.concat(data, axis=1).loc['1962':]

In [192]:
merged

,BAA,AAA,10 year,1 year,corp_bond_return,BBKMGDP,CPIAUCSL,20y_return,rf,30y_return,90day,mkt,ISM,SL,SM,SH,BL,HM,BH
1962-01-31,5.08,4.42,4.10,3.29,NaN,5.031371,30.040,-0.001602,0.002348,0.004844,0.002496,-0.036147,60.9,-2.6050,-0.8629,2.2592,-5.0016,-1.9027,0.3934
1962-02-28,5.07,4.42,4.00,3.21,NaN,9.228796,30.110,0.007828,0.001989,0.003634,0.002243,0.019512,61.1,0.7054,0.7314,1.7967,1.8606,2.4060,2.4047
1962-03-31,5.04,4.39,3.86,2.97,NaN,4.330766,30.170,0.017130,0.002007,0.026249,0.002244,-0.004688,60.6,0.4216,-1.1191,-0.5598,-0.3751,-0.2782,-2.1517
1962-04-30,5.02,4.33,3.86,3.07,NaN,2.061945,30.210,0.019020,0.002278,0.016849,0.002475,-0.063435,55.1,-6.3111,-7.2753,-6.8347,-6.6656,-5.5658,-6.0788
1962-05-31,5.00,4.28,3.90,2.99,NaN,2.304132,30.240,0.010481,0.002296,-0.004814,0.002465,-0.084620,52.2,-12.2520,-10.3050,-9.5501,-9.0826,-6.7701,-6.2353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-30,6.07,4.90,3.68,4.74,3986.08,-0.144042,298.598,0.071208,0.002856,0.077931,0.003114,0.058064,49.0,-0.4429,3.0537,2.6164,5.1322,5.4544,4.8403
2022-12-31,5.59,4.43,3.88,4.73,3947.76,-1.266861,298.990,-0.025772,0.003379,-0.035953,0.003280,-0.040356,48.4,-5.8863,-6.7824,-5.4729,-7.1720,-3.9805,-4.9431
2023-01-31,5.50,4.40,3.52,4.68,4230.53,NaN,300.536,NaN,NaN,NaN,NaN,NaN,47.4,14.0186,11.0995,8.1850,7.4981,5.5366,5.1740
2023-02-28,5.59,4.56,3.92,5.02,4005.19,NaN,301.648,NaN,NaN,NaN,NaN,NaN,47.7,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
# merged.to_csv('merged_data')

## GDP Analysis

In [210]:
# Reindex GDP to end of quarter
gdp_data.index = gdp_data.index.shift(2, freq='M')

In [202]:
# End of quarter cummulative GDP from monthly growth rates
gdp_bbk = ((bbk_data['BBKMGDP'].loc['1960':].dropna()/100) + 1).cumprod().resample('Q').last()

In [235]:
gdp = pd.merge(gdp_bbk, gdp_data, left_index=True, right_index=True)

In [236]:
gdp = np.log(gdp/gdp.shift(1)).dropna()
gdp['GDPC1'] = gdp['GDPC1']*10

In [237]:
import statsmodels.api as sm

In [238]:
gdp

,BBKMGDP,GDPC1
DATE,,
1960-06-30,-0.055851,-0.054053
1960-09-30,0.037754,0.048823
1960-12-31,-0.187642,-0.129171
1961-03-31,0.184926,0.067264
1961-06-30,0.207384,0.168319
...,...,...
2021-12-31,0.161012,0.168148
2022-03-31,-0.087579,-0.041118
2022-06-30,0.010741,-0.014474


In [257]:
mod = sm.OLS(gdp['BBKMGDP'], gdp['GDPC1']).fit()

In [258]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                BBKMGDP   R-squared (uncentered):                   0.894
Model:                            OLS   Adj. R-squared (uncentered):              0.893
Method:                 Least Squares   F-statistic:                              2102.
Date:                Tue, 14 Mar 2023   Prob (F-statistic):                   1.11e-123
Time:                        12:06:55   Log-Likelihood:                          379.77
No. Observations:                 251   AIC:                                     -757.5
Df Residuals:                     250   BIC:                                     -754.0
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
GDPC1          1.1745      0.026     45.845      0.000       1.124       1.225
==============================================================================
Omnibus:                      299.352   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            25503.705
Skew:                          -4.876   Prob(JB):                         0.00
Kurtosis:                      51.410   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-squared of 89.4% suggests that aggregated to quarterly, the monthly estimates are very good.